In [49]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import DMatrix

In [50]:
#import of complete dataframe
rawdata = pd.read_csv(r"D:\Sciebo\TechLabs_AI_Track\Team_11\Data\data_rhein.csv")
rawdata.head()

,Unnamed: 0,Kürzel,Status Nr.,Bemerkung,Labornummer,Jahr,Datum,Dauer,Mittl.Abfluss,Abflussumme,...,Quecksilber,NP,NP1EO,NP2EO,NP3EO,Calcium (IC),Status Nr.,year,month,day
0,0,RE,7,NaN,16354.0,1975.0,1975-05-26 11:15:00,168.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975,5,26
1,1,RE,1,NaN,16487.0,1975.0,1975-06-02 11:45:00,169.0,683.0,415.5372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975,6,2
2,2,RE,4,Pb unsicher: Wartung oder Beginn von Hochwasser?,16619.0,1975.0,1975-06-09 10:40:00,167.0,680.0,408.8160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975,6,9
3,3,RE,1,NaN,16705.0,1975.0,1975-06-16 11:20:00,169.0,609.0,370.5156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975,6,16
4,4,RE,1,NaN,16843.0,1975.0,1975-06-23 13:30:00,170.0,735.0,449.8200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975,6,23


In [51]:
#pick one station and one value (Nitrat)
dataRE = rawdata[["Kürzel","Nitrat","year","month","day"]]
dataRE.head(2)

,Kürzel,Nitrat,year,month,day
0,RE,0.4,1975,5,26
1,RE,0.7,1975,6,2


In [52]:
data = dataRE.loc[rawdata["Kürzel"]=="RE"]
data = data.drop("Kürzel", axis=1)
data.head(2)

,Nitrat,year,month,day
0,0.4,1975,5,26
1,0.7,1975,6,2


In [53]:
#calculate split for train/test data
number_observations = data.shape[0]
index = int(0.8 * number_observations)
print("Measurements:", number_observations)
print("80 % of value Nb.",index)

Measurements: 1264
80 % of value Nb. 1011


In [54]:
#definde target
target=data["Nitrat"]

In [55]:
# define data split and labels
test_data = data.iloc[index:,:]
test_label = target.iloc[index:]

train_data = data.iloc[:index,:]
train_label = target.iloc[:index]

In [41]:
#define matrix with data and label
train_matrix = DMatrix(data=train_data, label=train_label)
test_matrix = DMatrix(data=test_data, label=test_label)

C:\Users\raiko\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\raiko\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [42]:
#define model
model = xgb.train(dtrain=train_matrix, params={"silent": False}, verbose_eval=True)
prediction = model.predict(test_matrix)

In [43]:
#create output: rsme of prediction
rmse = np.sqrt(np.mean((test_label - prediction) ** 2))
print("RMSE: {}".format(rmse))

RMSE: 0.7686699707937081
